In [2]:
import os
import os.path as osp
import time
import datetime
import random
from PIL import Image
import numpy as np
from sklearn.metrics import accuracy_score
import torch.nn as nn
from torch.autograd import Function
import torch.optim as optim
import torch.nn.functional as F
import tensorflow as tf
import torch
import pandas as pd
import torch.utils.data as data
from torch.utils.data import Dataset, TensorDataset, DataLoader, RandomSampler, SequentialSampler, WeightedRandomSampler
import logging
from torchvision import datasets,transforms, models
from tqdm.notebook import tqdm
import glob
import matplotlib.image as image
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [3]:
#TRAIN_DATA
train_data = np.array([])
for r, dirnames, fn in os.walk('../input/hsgs-hackathon2022/train_data/Train'):
    for dirname in sorted(dirnames):
        print(dirname)
        cur_path = '../input/hsgs-hackathon2022/train_data/Train' + '/' + dirname
        for filename in sorted(os.listdir(cur_path)):
            train_data = np.append(train_data, os.path.join(cur_path, filename))

In [3]:

# #show a Image
# img = Image.open(train_data[21660])
# transform = transforms.Resize((224, 224))
# resized_image = transform(img)

# resized_image
# # Convert a image path from list to matrix
# # im = image.imread(train_data[20000])
# # im.shape

In [4]:
# TRAIN_LABEL
train_label = np.array([])
curDir = '../input/hsgs-hackathon2022/train_data/Train_labels'
for file in sorted(os.listdir(curDir)):
    datas = pd.read_csv(f'{os.path.join(curDir, file)}').values
    label_data = datas[:, 1]
    train_label = np.concatenate((train_label, label_data), axis = 0)

In [10]:
IM_SIZE = 224
BATCH_SIZE = 32

In [42]:
Transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.RandomHorizontalFlip(0.5),
    transforms.Resize((IM_SIZE, IM_SIZE)),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

In [43]:
class GetData(Dataset):
    def __init__(self, trainData, trainLabel, transform = None, isTrain = True):
        self.trainData = trainData
        self.trainLabel = trainLabel
        self.transform = transform
        self.isTrain = isTrain
    
    def __len__(self):
        return len(self.trainData)
    
    def __getitem__(self, idx):
        img_path = self.trainData[idx]
        img = Image.open(img_path)
        
        if self.transform:
            img_transformed = self.transform(img)
        
        if self.isTrain:
            label = self.trainLabel[idx]
            return img_transformed, label
        else:
            return img_transformed

In [95]:
sum(train_label)

In [19]:
train_XX = []
val_XX = []
train_YY = []
val_YY = []

cnt_x = 2000
cnt_y = 2500
idx = 0

used = []
for _ in range(len(train_data)):
    used.append(False)

while True:
    if cnt_x == 0 and cnt_y == 0:
        break
    
    if cnt_x != 0 and train_label[idx] == 1:
        train_XX.append(train_data[idx])
        train_YY.append(1)
        cnt_x -= 1
        used[idx] = True
    if cnt_y != 0 and train_label[idx] == 0:
        train_XX.append(train_data[idx])
        train_YY.append(0)
        cnt_y -= 1
        used[idx] = True
    idx += 1

cnt_x = 1000
cnt_y = 1000
idx = 0

while True:
    if cnt_x == 0 and cnt_y == 0:
        break
    if used[idx]:
        idx += 1
        continue
    if cnt_x != 0 and train_label[idx] == 1:
        val_XX.append(train_data[idx])
        val_YY.append(1)
        cnt_x -= 1
    if cnt_y != 0 and train_label[idx] == 0:
        val_XX.append(train_data[idx])
        val_YY.append(0)
        cnt_y -= 1
    idx += 1

In [20]:
train_dataloader = DataLoader(GetData(train_XX, train_YY, Transform), batch_size = BATCH_SIZE)
val_dataloader = DataLoader(GetData(val_XX, val_YY, Transform), batch_size = BATCH_SIZE)

In [21]:
len(val_dataloader)

In [62]:
train_X, val_X, train_Y, val_Y = train_test_split(train_data[:6000], train_label[:6000], test_size = 0.2, random_state = 42)
train_dataloader = DataLoader(GetData(train_X, train_Y, Transform), batch_size = BATCH_SIZE, shuffle = True)
val_dataloader = DataLoader(GetData(val_X, val_Y, Transform), batch_size = BATCH_SIZE, shuffle = True)

In [22]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [8]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        # 3x224x224 => 32x222x222
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
        # 32x222x222 => 32x111x111
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        # 32x111x111 => 64x111x111
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding='same')
        # 64x111x111 => 64x55x55
        self.avgpool = nn.AvgPool2d(kernel_size=2, stride=2)
        self.linear1 = nn.Linear(64*55*55, 32)
        self.linear2 = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()
  
    def forward(self, x):
        x = self.conv1(x)
#         print(x.size())
        x = self.maxpool1(x)
#         print(x.size())
        x = self.conv2(x)
#         print(x.size())
        x = self.avgpool(x)
#         print(x.size())
        x = torch.flatten(x, 1)
#         print(x.size())
        x = self.linear1(x)
#         print(x.size())
        x = self.linear2(x)
#         print(x.size())
        x = self.sigmoid(x)
        return x

# Model 1

In [9]:
class ConvBlock(nn.Module):
    def __init__(self, in_fts, out_fts, k, s, p, bias=False):
        super(ConvBlock, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels=in_fts, out_channels=out_fts, kernel_size=k, stride=s, padding=p, bias=bias),
            nn.BatchNorm2d(out_fts),
            nn.ReLU(inplace=True)
        )
        
    def forward(self, inp_img):
        return self.conv(inp_img)

class MyModel(nn.Module):
    def __init__(self, in_fts=3):
        super(MyModel, self).__init__()
        self.conv1 = ConvBlock(in_fts,64,7,2,3)
        self.conv2 = ConvBlock(64,128,3,1,1)
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = ConvBlock(128,256,3,1,1)
        self.conv4 = ConvBlock(256,512,3,1,1)
        self.conv5 = ConvBlock(512,512,3,1,1)
        
        self.avgpool = nn.AdaptiveAvgPool2d(output_size=(1,1))
        self.fc = nn.Sequential(
            nn.Linear(512,300),
            nn.Dropout(0.5),
            nn.Linear(300,300),
            nn.Dropout(0.5),
            nn.Linear(300,1),
            nn.Sigmoid()
        )
        

    def forward(self, input_img):
        N = input_img.shape[0]
        x = self.conv1(input_img)
        x = self.maxpool(x)
        x = self.conv2(x)
        x = self.maxpool(x)
        x = self.conv3(x)
        x = self.maxpool(x)
        x = self.conv4(x)
        x = self.maxpool(x)
        x = self.conv5(x)
        x = self.avgpool(x)
        x = torch.flatten(x,1)
        x = self.fc(x)
        
        return x

In [13]:
def Training(train_loader, val_loader):
    EPOCHS = 3
    for epoch in range(3):
        #Train
        total_loss_train = 0
        total_acc_train = 0
        
        loop = tqdm(train_loader)
        loop.set_description(f"{epoch + 1} / {EPOCHS}")
        
        count = 1
        
        for x, y in loop:
            x = x.to(device)
            y = y.to(device)
            
            output = model(x.float())
            batch_loss = criterion(torch.squeeze(output), y.float())
            total_loss_train += batch_loss.item()
            
            pred = []
            
            for i in output:
                if i.item() >= 0.5:
                    pred.append(1)
                else:
                    pred.append(0)

            accuracy = torch.mean((torch.tensor(pred).to(device) == y).float())
            total_acc_train += accuracy.item()

            optimizer.zero_grad()
            batch_loss.backward()
            optimizer.step()

            loop.set_postfix({'Train Loss': total_loss_train / count,'Train Accuracy': total_acc_train / count})
            count += 1
        
        #Validation
        
        iteration = 1
        loop = tqdm(val_loader)
        total_val_loss, total_val_accuracy = 0,0
        model.eval()
        with torch.no_grad():
            for img,label in loop:
                img = img.to(device)
                label = label.to(device)
                output = model(img)

                L = criterion(output, label.float().unsqueeze(1))
                total_val_loss += L.item()

                pred = []
                for i in output:
                    if i.item() >= 0.5:
                        pred.append(1)
                    else:
                        pred.append(0)

                accuracy = torch.mean((torch.tensor(pred).to(device) == label).float())
                total_val_accuracy += accuracy.item()

                loop.set_postfix({'Val Loss':total_val_loss/iteration,'Val Accuracy':total_val_accuracy/iteration})
                iteration += 1
        model.train()
        scheduler.step(total_val_loss/iteration)

In [15]:
model = MyModel()
LR = 1e-1
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=1, verbose=True)

criterion = nn.BCELoss()
# optimizer = torch.optim.Adam(model.parameters())
optimizer = torch.optim.SGD(model.parameters(), lr = LR)

model = model.to(device)
criterion = criterion.to(device)



Training(train_dataloader, val_dataloader)

# kf = KFold(n_splits = 3, random_state = 42, shuffle = True)

# for train_index, val_index in kf.split(train_data[:4500]):
#     train_X = []
#     train_Y = []
#     for idx in train_index:
#         train_X.append(train_data[idx])
#         train_Y.append(train_label[idx])
    
#     val_X = []
#     val_Y = []
#     for idx in val_index:
#         val_X.append(train_data[idx])
#         val_Y.append(train_label[idx])
    
#     train_dataloader = DataLoader(GetData(train_X, train_Y, Transform), batch_size = BATCH_SIZE)
#     val_dataloader = DataLoader(GetData(val_X, val_Y, Transform), batch_size = BATCH_SIZE)
#     Training(train_dataloader, val_dataloader)

In [78]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print (name, param.data)
    break

In [38]:
save_path = "MyModel.pt"
torch.save(model, save_path)

# Model 2 (VGG)

In [8]:
class MyVGG(nn.Module):
    def __init__(self):
        super(MyVGG, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 64, kernel_size = 3, padding = 'same')
        self.conv2 = nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = 3, padding = 'same')
        self.conv3 = nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 3, padding = 'same')
        self.conv4 = nn.Conv2d(in_channels = 128, out_channels = 128, kernel_size = 3, padding = 'same')
        self.conv5 = nn.Conv2d(in_channels = 128, out_channels = 256, kernel_size = 3, padding = 'same')
        self.conv6 = nn.Conv2d(in_channels = 256, out_channels = 256, kernel_size = 3, padding = 'same')
        self.conv7 = nn.Conv2d(in_channels = 256, out_channels = 512, kernel_size = 3, padding = 'same')
        self.conv8 = nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = 3, padding = 'same')
        self.maxpool = nn.MaxPool2d(kernel_size = 2, stride = 2)
        self.relu = nn.ReLU(inplace = True)
        self.linear1 = nn.Linear(7*7*512, 4096)
        self.linear2 = nn.Linear(4096, 4096)
        self.linear3 = nn.Linear(4096, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
#         print(x.shape)
        x = self.conv2(x)
        x = self.relu(x)
#         print(x.shape)
        x = self.maxpool(x)
#         print(x.shape)
        x = self.conv3(x)
        x = self.relu(x)
#         print(x.shape)
        x = self.conv4(x)
        x = self.relu(x)
#         print(x.shape)
        x = self.maxpool(x)
#         print(x.shape)
        x = self.conv5(x)
        x = self.relu(x)
#         print(x.shape)
        x = self.conv6(x)
        x = self.relu(x)
#         print(x.shape)

        x = self.conv6(x)
        x = self.relu(x)
#         print(x.shape)
        x = self.maxpool(x)
#         print(x.shape)
        x = self.conv7(x)
        x = self.relu(x)
#         print(x.shape)
        x = self.conv8(x)
        x = self.relu(x)
#         print(x.shape)
        x = self.conv8(x)
        x = self.relu(x)
#         print(x.shape)
        x = self.maxpool(x)
#         print(x.shape)
        x = self.conv8(x)
        x = self.relu(x)
#         print(x.shape)
        x = self.conv8(x)
        x = self.relu(x)
#         print(x.shape)
        x = self.conv8(x)
        x = self.relu(x)
#         print(x.shape)
        x = self.maxpool(x)
#         print(x.shape)
        x = torch.flatten(x, 1)
#         print(x.shape)
        x = self.linear1(x)
        x = self.relu(x)
#         print(x.shape)
        x = self.linear2(x)
        x = self.relu(x)
#         print(x.shape)
        x = self.linear3(x)
#         print(x.shape)
        x = self.sigmoid(x)
        return x

In [9]:
def Training_VGG(train_loader, val_loader, model_def, loss_fn):
    EPOCHS = 3
    for epoch in range(3):
        #Train
        total_loss_train = 0
        total_acc_train = 0
        
        loop = tqdm(train_loader)
        loop.set_description(f"{epoch + 1} / {EPOCHS}")
        
        count = 1
        
        for x, y in loop:
            x = x.to(device)
            y = y.to(device)
            
            output = model_def(x.float())
            batch_loss = loss_fn(torch.squeeze(output), y.float())
            total_loss_train += batch_loss.item()
            
            pred = []
            
            for a in output:
                if a.item() >= 0.5:
                    pred.append(1)
                else:
                    pred.append(0)

            accuracy = torch.mean((torch.tensor(pred).to(device) == y).float())
            total_acc_train += accuracy.item()

            optimizer.zero_grad()
            batch_loss.backward()
            optimizer.step()

            loop.set_postfix({'Train Loss': total_loss_train / count,'Train Accuracy': total_acc_train / count})
            count += 1
        
        #Validation
        
        iteration = 1
        loop = tqdm(val_loader)
        total_val_loss, total_val_accuracy = 0,0
        model_def.eval()
        with torch.no_grad():
            for img,label in loop:
                img = img.to(device)
                label = label.to(device)
                output = model_def(img)

                L = loss_fn(output, label.float().unsqueeze(1))
                total_val_loss += L.item()

                pred = []
                for a in output:
                    if a.item() >= 0.5:
                        pred.append(1)
                    else:
                        pred.append(0)

                accuracy = torch.mean((torch.tensor(pred).to(device) == label).float())
                total_val_accuracy += accuracy.item()

                loop.set_postfix({'Val Loss':total_val_loss/iteration,'Val Accuracy':total_val_accuracy/iteration})
                iteration += 1
        model_def.train()
        scheduler.step(total_val_loss/iteration)

In [5]:
model_vgg = MyVGG()
LR = 1e-1
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
optimizer = torch.optim.SGD(model_vgg.parameters(), lr = LR)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=1, verbose=True)

criterion = nn.BCELoss()
# optimizer = torch.optim.Adam(model.parameters())

model_vgg = model_vgg.to(device)
criterion = criterion.to(device)
Training_VGG(train_dataloader, val_dataloader, model_vgg, criterion)

# Model ResNet


In [24]:
import torchvision
from torchinfo import summary

In [23]:
!pip install torchinfo

In [50]:
pre_trained = torchvision.models.resnet18(pretrained=True)
pre_trained = pre_trained.to(device)
# for param in pre_trained.parameters():
#     param.requires_grad = False
pre_trained

In [66]:
for name, param in pre_trained.fc.named_parameters():
    print(param.requires_grad)

In [63]:
net = nn.Sequential(
    pre_trained,
    nn.Dropout(0.2),
    nn.Linear(in_features = 1000, out_features = 256),
    nn.Dropout(0.2),
    nn.Linear(in_features = 256,  out_features = 1),
    nn.Sigmoid()
)

In [28]:
net

In [64]:
for name, param in net[0].named_parameters():
    print(param)

In [64]:
LR = 1e-2
criterion_res = nn.BCELoss()
optimizer_res = torch.optim.SGD(net.parameters(), lr = LR, momentum = 0.9)
scheduler_res = optim.lr_scheduler.ReduceLROnPlateau(optimizer_res, patience=1, verbose=True)

In [65]:
n_epochs = 5
total_step = len(train_dataloader)

val_loss = []
val_acc = []
train_loss = []
train_acc = []

for epoch in range(1, n_epochs + 1):
    total_train_loss = 0.0
    total_train_acc = 0.0
    
    count = 1
    loop = tqdm(train_dataloader)
    loop.set_description(f"{epoch}/{n_epochs}")
    
    for (x, y) in loop:
        x = x.to(device)
        y = y.to(device)
        optimizer_res.zero_grad()
        
        output = net(x)
        loss = criterion_res(output, y.float().unsqueeze(1))
        loss.backward()
        optimizer_res.step()
        
        total_train_loss += loss.item()
        
        pred = []
        for i in output:
            if i.item() >= 0.5:
                pred.append(1)
            else:
                pred.append(0)
        
        acc = torch.mean((torch.tensor(pred).to(device) == y).float())
        total_train_acc += acc.item()
        
        loop.set_postfix({'Train Loss:': total_train_loss / count, 'Train Accuracy:': total_train_acc / count})
        count += 1
    
    train_acc.append(total_train_acc / count)
    train_loss.append(total_train_loss / count)
    
    
    #Validation
    
    count = 1
    total_val_acc = 0
    total_val_loss = 0
    
    loop = tqdm(val_dataloader)
    
    with torch.no_grad():
        net.eval()
        for x, y in loop:
            x = x.to(device)
            y = y.to(device)
            output_t = net(x)
            loss_t = criterion_res(output_t, y.float().unsqueeze(1))
            total_val_loss += loss_t.item()
            
            pred = []
            for i in output_t:
                if i.item() >= 0.5:
                    pred.append(1)
                else:
                    pred.append(0)
            
            acc = torch.mean((torch.tensor(pred).to(device) == y).float())
            total_val_acc += acc.item()
            
            loop.set_postfix({'Val Loss:': total_val_loss / count, 'Val Accuracy:': total_val_acc / count})
            count += 1
            
        val_acc.append(total_val_acc/count)
        val_loss.append(total_val_loss/count)
    net.train()
    scheduler_res.step(total_val_loss/count)
    
    save_myModel = f"MyModel{epoch}.pt"
    torch.save(net, save_myModel)


In [38]:
a = torch.Tensor([3,4,5])
b = torch.Tensor([4,5,5])
torch.sum(a==b).item()

In [35]:
save_path = "MyResNet3.pt"
torch.save(net, save_path)

# Test

In [36]:
test_data = np.array([])
test_data_path = []
for r, dirnames, fn in os.walk('../input/hsgs-hackathon2022/Test_data/Test'):
    for dirname in sorted(dirnames):
        print(dirname)
        cur_path = '../input/hsgs-hackathon2022/Test_data/Test' + '/' + dirname
        for filename in sorted(os.listdir(cur_path)):
            test_data = np.append(test_data, os.path.join(cur_path, filename))
            test_data_path.append(dirname + '_' + filename[:-4])

In [37]:
test_dataset = GetData(test_data, _, Transform, False)
test_dataloader = DataLoader(test_dataset, batch_size = BATCH_SIZE)

In [ ]:
for idx in range(1, 6):
    test_model = torch.load(f'../working/MyModel{idx}.pt')
    test_model = test_model.to(device)
    test_model.eval()
    loop = tqdm(test_dataloader)
    test_pred = []
    
    with torch.no_grad():
        for img in loop:
            img = img.to(device)
            output = temp_model(img)

            for i in output:
                if i.item() >= 0.5:
                    test_pred.append(1)
                else:
                    test_pred.append(0)
    submission = pd.DataFrame(data = zip(test_data_path, test_pred), columns = ["Frame", "Label"])
    submission.to_csv(f"sub{idx}.csv", index=False)

In [38]:
temp_model = torch.load("../working/MyResNet3.pt")
temp_model = temp_model.to(device)
temp_model.eval()
loop = tqdm(test_dataloader)
test_pred = []

with torch.no_grad():
    for img in loop:
        img = img.to(device)
        output = temp_model(img)
        
        for i in output:
            if i.item() >= 0.5:
                test_pred.append(1)
            else:
                test_pred.append(0)

In [39]:
len(test_pred)

In [40]:
submission = pd.DataFrame(data = zip(test_data_path, test_pred), columns = ["Frame", "Label"])
submission.to_csv("sub.csv", index=False)